# Metropolis-Hastings Algorithm (Metropolis)

Metropolis algorithm is a markov chain monte carlo simulation that will converge to or near our desired solution in the long run.

To apply the Metropolis algorithm we need the following:

1.   A defined state space
2.   An energy function defined on the states
3.   An idea of neighboring states





## Tools

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import yfinance as yf
import cvxpy as cp
import warnings
warnings.filterwarnings('ignore')

## Algorithm / Functions

### Metropolis

In [2]:
# Calculating the Variance
# Var(x) = x^T V  x
# Constraints can be added to the energy (such as no short selling)
def energy(x, sigma, expected_returns, target_return):
    # No shorting by making energy penalty extremely high for any negative weights
    if (x < 0).sum() > 0:
      return 1000
    # Requiring a certain expected return
    if ((x @ expected_returns.T) < target_return):
      return 500

    return x.T @ sigma @ x

# Metropolis
def metropolis_minimum_variance(sigma, expected_returns, target_return):

    statement = '''

    Temperature should be 0 if there are no constraints
    If there are constraints you can trial and error for the most efficient temperature given the constraints

    Number of iterations - the larger the better

    If final energy/variance is 1000, it means there are likely no solutions that meet the constraints


    '''

    print(statement)


    # temp -> inf: uniform on all the states (explores every possibility)
    # temp -> 0: uniform on stable states (local minimums and minimums)
    temp = float(input("Enter the temperature: "))
    N = int(input("Enter the number of iterations: "))
    N10 = int(N/10)



    number_of_stocks = sigma.shape[0]

    # Random starting weight using standard Gaussian
    x = np.random.randn(number_of_stocks)
    x = np.abs(x)
    # Normalize sum to 1
    x /= x.sum()

    current_variance = energy(x, sigma, expected_returns, target_return)

    # Displaying initial weights
    print("\n")
    print("=" * 80)
    print("Initial Variance: ", current_variance)
    print("Intial Weights")
    display(x)
    print("=" * 80)


    best_x = x
    best_variance = current_variance

    t = 0


    while (t < N):
        accept = 0
        x_cand = x.copy()
        x_change = np.random.uniform(-0.5, 0.5)
        x1, x2 = np.random.choice(range(number_of_stocks), size = 2, replace = False)
        x_cand[x1] += x_change
        x_cand[x2] -= x_change
        cand_variance = energy(x_cand, sigma, expected_returns, target_return)

        delta = cand_variance - current_variance

        if (delta < 0):
            accept = 1
        elif (temp > 0):
            prob = np.exp(- delta / temp)
            accept = np.random.rand() < prob

        if (accept):
            x = x_cand
            current_variance = cand_variance

            # Keeping track of best result so far
            if (cand_variance < best_variance):
                best_x = x_cand
                best_variance = cand_variance

        # Report the progress periodically
        if (t%N10 == 0):
            print(f"Still thinking... on iteration {t}")
            print(f"Current variance is: {current_variance} \n")

        t+=1

    # Metropolis results

    best_annual_volatility = np.sqrt(best_variance) * np.sqrt(252)

    print("=" * 80)

    print("Metropolis Minimum Variance Found is : ", best_variance)
    print("Annual Volatility of Portfolio: ", best_annual_volatility)
    print("Expected Return: ", (1+x @ expected_returns.T)**252 - 1)
    print("-" * 80)

    w = pd.DataFrame(best_x, index = tickers, columns = ['Weight'])
    display(w.sort_values(by = 'Weight', ascending = False))

    print("=" * 80)



    return best_x, best_variance


### CVXPY

Using CVXPY, which is much faster than Metropolis, to find the optimal portfolio

In [3]:
def cvx_minimum_variance(covariance_matrix, expected_returns, target_return):
  number_of_stocks = covariance_matrix.shape[0]
  x = cp.Variable(number_of_stocks)

  sigma = cp.Parameter((number_of_stocks, number_of_stocks), PSD=True)
  sigma.value = covariance_matrix

  portfolio_variance = cp.quad_form(x, sigma)
  portfolio_return = x @ expected_returns.T

  objective = cp.Minimize(portfolio_variance)

  constraints = [
    cp.sum(x) == 1,
    x >= 0,
    portfolio_return >= target_return
  ]


  problem = cp.Problem(objective, constraints)

  problem.solve()

  best_variance = x.value.T @ covariance_matrix @ x.value
  best_annual_volatility = np.sqrt(best_variance) * np.sqrt(252)


  print("=" * 80)
  print("CVX Minimum Variance Found is : ", best_variance)
  print("Annual Volatility of Portfolio: ", best_annual_volatility)
  print("Expected Return: ", (1 + x.value @ expected_returns.T) ** 252 - 1)
  print("-" * 80)
  x = pd.DataFrame(x.value.round(6), index = tickers, columns = ['Weight'])
  display(x.sort_values(by = 'Weight', ascending = False))
  print("=" * 80)

  return x

### Other Functions

In [4]:
def geometric_mean(returns_data):
  growth_factors = returns_data + 1
  geometric_mean = growth_factors.prod() ** (1/len(growth_factors)) - 1
  return geometric_mean

## Data

In [5]:
# Getting Data from Yahoo Finance
tickers = ['PLTR', 'INTC', 'MP', 'SOUN', 'HPE', 'CAVA', 'BA', 'NET', 'NVDA']
data = yf.download(tickers, start='2024-06-01', end='2025-06-01')
data_close = data['Close']

[*********************100%***********************]  9 of 9 completed


In [6]:
# Getting the returns
returns_data = data_close.pct_change().dropna()
print("=" * 80)
print("Returns of Stocks")
print("-" * 80)
display(returns_data)
print("=" * 80)

Returns of Stocks
--------------------------------------------------------------------------------


Ticker,BA,CAVA,HPE,INTC,MP,NET,NVDA,PLTR,SOUN
Date,,,,,,,,,
2024-06-04,0.021943,0.003864,-0.019499,-0.008584,-0.044637,0.008750,0.012496,0.036585,0.031381
2024-06-05,0.006521,0.041884,0.106818,0.024975,0.001298,0.026904,0.051556,0.038914,0.010142
2024-06-06,0.008270,-0.076054,0.005647,-0.011696,-0.011666,0.020043,-0.011777,0.036150,-0.028112
2024-06-07,-0.006164,0.000000,0.022461,0.010519,-0.026885,-0.007298,-0.000909,-0.020177,-0.039256
2024-06-10,-0.000631,0.043862,0.027958,0.005530,0.010108,0.013997,0.007461,-0.007722,0.025806
...,...,...,...,...,...,...,...,...,...
2025-05-23,-0.005162,-0.011024,-0.007407,-0.024331,-0.019422,-0.001075,-0.011594,0.008341,0.003141
2025-05-27,-0.006572,-0.013065,0.029851,0.024938,-0.046724,0.023538,0.032066,0.000649,0.160752
2025-05-28,0.002338,-0.005222,-0.013935,-0.008759,0.044219,0.001360,-0.005092,0.002999,-0.013489


In [7]:
# Covariance Matrix
covariance_matrix = returns_data.cov()
print("=" * 80)
print("Covariance Matrix: ")
print("-" * 80)
display(covariance_matrix)
print("=" * 80)

Covariance Matrix: 
--------------------------------------------------------------------------------


Ticker,BA,CAVA,HPE,INTC,MP,NET,NVDA,PLTR,SOUN
Ticker,,,,,,,,,
BA,0.000605,0.000326,0.000310,0.000385,0.000263,0.000331,0.000349,0.000371,0.000584
CAVA,0.000326,0.001309,0.000539,0.000416,0.000353,0.000545,0.000701,0.000847,0.000864
HPE,0.000310,0.000539,0.000915,0.000504,0.000327,0.000453,0.000612,0.000561,0.000802
INTC,0.000385,0.000416,0.000504,0.001589,0.000468,0.000348,0.000613,0.000636,0.000826
MP,0.000263,0.000353,0.000327,0.000468,0.001877,0.000245,0.000326,0.000590,0.000654
NET,0.000331,0.000545,0.000453,0.000348,0.000245,0.001065,0.000624,0.000662,0.000821
NVDA,0.000349,0.000701,0.000612,0.000613,0.000326,0.000624,0.001381,0.000839,0.001071
PLTR,0.000371,0.000847,0.000561,0.000636,0.000590,0.000662,0.000839,0.002108,0.001424
SOUN,0.000584,0.000864,0.000802,0.000826,0.000654,0.000821,0.001071,0.001424,0.005550


## Calculation and Results

Metropolis example

In [8]:
geometric_returns = geometric_mean(returns_data)
target = float(input("Enter the annual target return in decimal (if 10%, enter 0.1): " ))
target_daily = (1 + target) ** (1/252) - 1
print("Target Daily Return: ", target_daily)


Enter the annual target return in decimal (if 10%, enter 0.1): .1
Target Daily Return:  0.0003782865315342665


In [9]:
# Call our Metropolis
w, w_var = metropolis_minimum_variance(covariance_matrix.copy(), geometric_returns.copy(), target_daily)


    
    Temperature should be 0 if there are no constraints
    If there are constraints you can trial and error for the most efficient temperature given the constraints

    Number of iterations - the larger the better
    
    If final energy/variance is 1000, it means there are likely no solutions that meet the constraints


    
Enter the temperature: 4
Enter the number of iterations: 500000


Initial Variance:  0.0007041984161320075
Intial Weights


array([0.02730309, 0.22131913, 0.05252363, 0.14255472, 0.18806381,
       0.16302922, 0.01725069, 0.12634239, 0.06161332])

Still thinking... on iteration 0
Current variance is: 0.0007387303905143532 

Still thinking... on iteration 50000
Current variance is: 0.0005649350008517121 

Still thinking... on iteration 100000
Current variance is: 0.0007399024810928669 

Still thinking... on iteration 150000
Current variance is: 0.0007439578403374923 

Still thinking... on iteration 200000
Current variance is: 0.0008486263406404832 

Still thinking... on iteration 250000
Current variance is: 0.0006386705343967294 

Still thinking... on iteration 300000
Current variance is: 0.0006597564772088805 

Still thinking... on iteration 350000
Current variance is: 0.0010337746098786577 

Still thinking... on iteration 400000
Current variance is: 0.0006984013731100918 

Still thinking... on iteration 450000
Current variance is: 0.0006372132902520893 

Metropolis Minimum Variance Found is :  0.00046776399371251917
Annual Volatility of Portfolio:  0.3433315109563275
Expected Return:  0.7419452416984416
------------------------

,Weight
PLTR,0.507175
MP,0.210907
CAVA,0.122264
INTC,0.070771
HPE,0.051734
NET,0.025346
SOUN,0.006957
NVDA,0.003382
BA,0.001462


pd.DataFrame(w * 3000, index = tickers)

CVXPY example

In [10]:
solution = cvx_minimum_variance(covariance_matrix.values.copy(), geometric_returns.values.copy(), target_daily)

CVX Minimum Variance Found is :  0.0004560837164824951
Annual Volatility of Portfolio:  0.33901784105499344
Expected Return:  0.21242990391282102
--------------------------------------------------------------------------------


,Weight
PLTR,0.489573
MP,0.172135
CAVA,0.136639
HPE,0.106297
INTC,0.058133
SOUN,0.037223
BA,0.000000
NET,0.000000
NVDA,0.000000


### Results

From the final portfolio output and its variance, we can conclude that although it is cool to use Metropolis for portfolio optimization, it is not practical compared to CVXPY or other optimization methods. However, Metropolis can come in handy when the constraints become so complex that there is no clear optimal solution (or to complex to find). In these situations, we can modify the Metropolis energy function to search for a "best possible" solution.